In [1]:
import os
import ee
import geemap
import time
import random
from datetime import datetime, timedelta , date
import ipywidgets as widgets
from IPython.display import display
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
#Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 
Map = geemap.Map()

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [5]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

In [6]:
# give the bands

band_name = widgets.Dropdown(
options=['ndvi'],
description='Band')

band_name

Dropdown(description='Band', options=('ndvi',), value='ndvi')

In [7]:
# gine the date frequency

frequency =widgets.Dropdown(
    options=['Year','Month'],
    value='Year',
    description='Frequency')

frequency

Dropdown(description='Frequency', options=('Year', 'Month'), value='Year')

In [8]:
# give the output floder and flie name
folder = FileChooser()
display(folder)


FileChooser(path='C:\Users\Yang Hsiu\Desktop\satellite python code\satellite python code\satellite python code…

In [9]:
# give the scale number
scale = widgets.IntSlider(
    value=1000,
    min=500,
    max=10000,
    step=100,
    description='Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

scale

IntSlider(value=1000, continuous_update=False, description='Scale:', max=10000, min=500, step=100)

In [10]:
# give CLOUDY_PIXEL_PERCENTAGE

CLOUDY_PIXEL_PERCENTAGE = widgets.IntSlider(
    value=20,
    min=0,
    max=100,
    step=10,
    description='CLOUDY_PIXEL_PERCENTAGE:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

CLOUDY_PIXEL_PERCENTAGE


IntSlider(value=20, continuous_update=False, description='CLOUDY_PIXEL_PERCENTAGE:', step=10)

In [11]:
roi = geemap.shp_to_ee(''.join(my_button.files)).geometry()

In [12]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def last_day_of_year(any_day):
    next_year = any_day.replace(year=any_day.year, month=12, day = 31)  # this will never fail
    return next_year

def yearlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.year < end.year:
            next_year = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_year = end
        if next_year == end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_year(begin).strftime("%Y-%m-%d")])
        begin = next_year
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result

In [13]:
#name of bands
inBands = ee.List(['QA60','B4','B8']);
outBands = ee.List(['QA60','red','nir']);
 
CloudCoverMax = 20



def maskS2clouds(image):
    qa = image.select('QA60');
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0));
    return image.updateMask(mask).divide(10000)

# #function to get the data
# def importData(studyArea,startDate,endDate):
 
#     # Get Sentinel-2 data
#     s2s =(ee.ImageCollection('COPERNICUS/S2')
#           .filter(ee.Filter.date(startDate,endDate))
#           .filterBounds(studyArea)
#           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',CloudCoverMax))
#           .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT',CloudCoverMax)))
    
#     #sentinel bands are in scale of 0.0001
#     s2s = s2s.select(inBands,outBands)
    
#     return s2s


In [14]:
# get indexes
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("ndvi")
    image = image.addBands(ndvi)

    return(image)

In [15]:
def geotif(time_list):
   
    for i in range(0,len(time_list)):
        star_time = time.time()
        
        dataset = ee.ImageCollection('COPERNICUS/S2') \
                        .filterDate(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1)) \
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUDY_PIXEL_PERCENTAGE.value)) \
                        .map(maskS2clouds) \
                        .map(getNDVI) \
                        .select(band_name.value) \
                        .mean()
        
        image = dataset.clip(roi).unmask()
        filename = os.path.join(os.path.expanduser(folder.selected) + '_{}.tif'.format(time_list[i]))
        geemap.ee_export_image(
            image, filename=filename, scale=scale.value, region=roi, file_per_band=False
                                )

In [16]:
if 'Year' in frequency.value.strip():
    time_list = yearlist(star.value, end.value)
    geotif(time_list)
else:
    pass
if "Month" in frequency.value.strip():
    time_list =monthlist(star.value, end.value)
    geotif(time_list)
else:
    pass

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Yang Hsiu\Desktop\plot\S222222_['2021-01-01', '2021-12-31'].tif
